In [1]:
import pandas as pd
import time
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd

In [6]:
query = "SELECT  ano,  id_municipio,   sigla_uf,  tipo_vinculo, sum(quantidade_vinculo) as quantidade_vinculos FROM  `basedosdados.br_ibge_munic.indicadores_quantidade_vinculo` where ano =2021 group by   1, 2, 3,4"

In [4]:
client = bigquery.Client()

In [7]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████| 27850/27850 [00:01<00:00, 16251.66rows/s]


In [8]:
df

,ano,id_municipio,sigla_uf,tipo_vinculo,quantidade_vinculos
0,2021,1100015,RO,Estatutários,1175
1,2021,1100023,RO,Estatutários,2191
2,2021,1100031,RO,Estatutários,250
3,2021,1100049,RO,Estatutários,1848
4,2021,1100056,RO,Estatutários,513
...,...,...,...,...,...
27845,2021,5222005,GO,Sem vínculo permanente,45
27846,2021,5222054,GO,Sem vínculo permanente,60
27847,2021,5222203,GO,Sem vínculo permanente,5
27848,2021,5222302,GO,Sem vínculo permanente,15


In [9]:
y=df.pivot_table(index={'id_municipio'},values='quantidade_vinculos', aggfunc=np.sum)

In [12]:
y = pd.DataFrame(y)
total = y.reset_index()


In [13]:
total=total.rename(columns={'quantidade_vinculos':'total_municipio'})

In [14]:
df1 = df.merge(total, right_on='id_municipio', left_on='id_municipio')

In [15]:
## Subindo para GBQ

dataset_ref = client.dataset('perfil_remuneracao') 

In [16]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27850 entries, 0 to 27849
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ano                  27850 non-null  Int64 
 1   id_municipio         27850 non-null  object
 2   sigla_uf             27850 non-null  object
 3   tipo_vinculo         27850 non-null  object
 4   quantidade_vinculos  27819 non-null  Int64 
 5   total_municipio      27850 non-null  Int64 
dtypes: Int64(3), object(3)
memory usage: 1.6+ MB


In [17]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('id_municipio','STRING',description='Identificador de município'),
  bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
bigquery.SchemaField('tipo_vinculo','STRING',description='Tipo de vinculo.'),
 bigquery.SchemaField('quantidade_vinculos','INTEGER',description='Quantidade de vinculos'),
 bigquery.SchemaField('total_municipio','INTEGER',description='Total de vínculos daquele municipio')
 ]

In [18]:
table_ref = dataset_ref.table('MUNIC_quantidade_vinculos_mapa') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=be591a1c-a44e-485b-ac5b-2c4798a564d9>